![example](images/director_shot.jpeg)

# Project Title

**Authors:** Rachel Fein
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

Microsoft want to make a movie that has the highest chances of success. The rate of sucess will be evaluated by the 
ROI (rate of investment) or NET profit, a formula of gross profit subtracted by budget. Specific atrubutes of the 
Movies with the highest ROI were analysised to give Microsoft the understanding of what common qualities at top
producing movie has. 

Top producing movies' runtime, release date, and budget were analysis. These three aspects were picked becuase 
they have shown to have a great impact on a movies' sucess and will help Microsoft develop a movie that will land 
in with the top. 

Finding and choosing the defining characteristics of a movie that help make it sucessful will give microsoft the 
best chance at sucess.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.

The data used for this project was from imdb (def of what imbd is)
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

Where did the data come from, and how do they relate to the data analysis questions?

To answer the data analysis question I need to find the hight ROI and to find that I need the 
total costs (budget and gross rev). Once I have the movies with the highest ROI I will look
into their genres, time of year released, and etc. The data I used was found in
imdb.title.basics.csv, tn.movie_budgets.csv, and bom.movie_gross.csv

The target variable is the titles with the highest ROI and those titles characteristics

In [92]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [93]:
# Import data needed for analysis after unzipping data using gzip -d filename.gz (starting with basics)
df_imdb = pd.read_csv('data/zippedData/imdb.title.basics.csv')
df_tn = pd.read_csv('data/zippedData/tn.movie_budgets.csv')

In [95]:
# find out what the data file consists of
# using .info tells us where we might be missing data
# which I will be using for my analysis does have missing data and this will have to be addressed. 
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [112]:
df_imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [116]:
df_imdb['primary_title'].nunique()

136071

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [89]:
# Here you run your code to clean the data
# We know that there is data we need that is a object datatype but should be an int 
# ex. budget and costs 

In [97]:
# editFor the intake and outcome files, I make them easier to work with by normalizing column names and dropping unnecessary columns.
# Start with putting release date in datetime format so we can extract the year which will be needed for inflation adjustment
# Also extract release month which will be needed for analysis when to release the movie
df_tn['release_date']= pd.to_datetime(df_tn['release_date'])
df_tn['release_year'] = pd.DatetimeIndex(df_tn['release_date']).year
df_tn['release_month'] = pd.DatetimeIndex(df_tn['release_date']).month
df_tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009,12
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,5
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,6
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,5
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017,12


In [99]:
# Next columns with money values must be changed from objects to integers to calculate net profit and for analysis 
# of budget
df_tn[df_tn.columns[3:6]] = df_tn[df_tn.columns[3:6]].replace('[\$,]', '', regex=True).astype(int)
df_tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_year       5782 non-null   int64         
 7   release_month      5782 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 361.5+ KB


In [100]:
# Next we will adjust production_budget & worldwide_gross for inflation so that that the movies can be evaluated fairly
# cpi is a python library that is widely used to account for inflation of the dollar. cpi.update insures we have the
# most up to date data the library holds
import cpi
cpi.update()

In [102]:
# Use a lambda function to account for inflation for movies made before 2019 the cpi library does not have inflation
# rates for 2020 or 2021 yet, however that is fine as the change in the dolar from 2019 to 2021 is not great EDIT
# Since movies released in 2019 or later arnt adjusted I used an else statement to place the orginal value in with
# the adjust value columns so I will not have missing data for movies in 2019 
df_tn["adjusted_production_budget"] = df_tn.apply(lambda x: cpi.inflate(x["production_budget"], x["release_year"]) 
                                                if (x["release_year"] < 2019) else x["production_budget"], axis=1) 
df_tn["adjusted_worldwide_gross"] = df_tn.apply(lambda x: cpi.inflate(x["worldwide_gross"], x["release_year"]) 
                                                if (x["release_year"] < 2019) else x["worldwide_gross"], axis=1) 
df_tn.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,adjusted_production_budget,adjusted_worldwide_gross
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,12,5.127072e+08,3.349300e+09
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,5,4.724294e+08,1.203123e+09
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,6,3.500000e+08,1.497624e+08
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,5,3.609991e+08,1.532023e+09
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,12,3.347058e+08,1.390266e+09


In [104]:
# calculated and created a column for net profit 
df_net_profit = df_tn.assign(net_profit = lambda x: df_tn['worldwide_gross'] 
                                    - df_tn['production_budget'])
df_net_profit.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,adjusted_production_budget,adjusted_worldwide_gross,net_profit
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2009,12,5.127072e+08,3.349300e+09,2351345279
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,5,4.724294e+08,1.203123e+09,635063875
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,2019,6,3.500000e+08,1.497624e+08,-200237650
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,5,3.609991e+08,1.532023e+09,1072413963
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,12,3.347058e+08,1.390266e+09,999721747


In [108]:
# Drop columns that are unncecessary for my analysis of budget, release date, & runtime 
# doing such will make the data easier to work with especially for data modeling 
df_net_profit.drop(columns = ['id', 'production_budget', 'domestic_gross', 'worldwide_gross'], inplace=True )

KeyError: "['id' 'production_budget' 'domestic_gross' 'worldwide_gross'] not found in axis"

In [110]:
df_net_profit.head()

,release_date,movie,release_year,release_month,adjusted_production_budget,adjusted_worldwide_gross,net_profit
0,2009-12-18,Avatar,2009,12,5.127072e+08,3.349300e+09,2351345279
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,2011,5,4.724294e+08,1.203123e+09,635063875
2,2019-06-07,Dark Phoenix,2019,6,3.500000e+08,1.497624e+08,-200237650
3,2015-05-01,Avengers: Age of Ultron,2015,5,3.609991e+08,1.532023e+09,1072413963
4,2017-12-15,Star Wars Ep. VIII: The Last Jedi,2017,12,3.347058e+08,1.390266e+09,999721747


In [123]:
# runtime data is located in df_imdb. I merged the runtime column to the df_net_profit table
# this df will only be used for data modeling of runtime as rows will get dropped doing an inner merge since only
# titles in both dataframes will remain once merged
# first make columns in both dataframes have same name for movie title
# movie_title is also a better name and helps with readablilty
df_net_profit.rename(columns={'movie': 'movie_title'}, inplace = True)
df_imdb.rename(columns={'primary_title': 'movie_title'}, inplace = True)
df_imdb.head()

,tconst,movie_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [132]:
# combine and drop columns that I dont need for data analysis
df_combined = df_net_profit.merge(df_imdb, on='movie_title', how= 'inner')
df_combined.drop(columns = ['tconst', 'original_title', 'start_year'], inplace=True )
df_combined.head()

,release_date,movie_title,release_year,release_month,adjusted_production_budget,adjusted_worldwide_gross,net_profit,runtime_minutes,genres
0,2009-12-18,Avatar,2009,12,5.127072e+08,3.349300e+09,2351345279,93.0,Horror
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,2011,5,4.724294e+08,1.203123e+09,635063875,136.0,"Action,Adventure,Fantasy"
2,2019-06-07,Dark Phoenix,2019,6,3.500000e+08,1.497624e+08,-200237650,113.0,"Action,Adventure,Sci-Fi"
3,2015-05-01,Avengers: Age of Ultron,2015,5,3.609991e+08,1.532023e+09,1072413963,141.0,"Action,Adventure,Sci-Fi"
4,2018-04-27,Avengers: Infinity War,2018,4,3.092040e+08,2.110971e+09,1748134200,149.0,"Action,Adventure,Sci-Fi"


In [133]:
# we see that there are rows that are missing runtimes 
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3815 entries, 0 to 3814
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   release_date                3815 non-null   datetime64[ns]
 1   movie_title                 3815 non-null   object        
 2   release_year                3815 non-null   int64         
 3   release_month               3815 non-null   int64         
 4   adjusted_production_budget  3815 non-null   float64       
 5   adjusted_worldwide_gross    3815 non-null   float64       
 6   net_profit                  3815 non-null   int64         
 7   runtime_minutes             3328 non-null   float64       
 8   genres                      3743 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 298.0+ KB


In [137]:
# I decided to drop the rows with no runtime data since the dataframe will only be used for runtime analysis
df_combined.dropna(subset=['runtime_minutes'], inplace=True)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3328 entries, 0 to 3814
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   release_date                3328 non-null   datetime64[ns]
 1   movie_title                 3328 non-null   object        
 2   release_year                3328 non-null   int64         
 3   release_month               3328 non-null   int64         
 4   adjusted_production_budget  3328 non-null   float64       
 5   adjusted_worldwide_gross    3328 non-null   float64       
 6   net_profit                  3328 non-null   int64         
 7   runtime_minutes             3328 non-null   float64       
 8   genres                      3309 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 260.0+ KB


In [139]:
# we have repeating movie title rows varing runtimes for the same title. Since the df_imdb data had multiple movie title rows for the same title. 
# I grouped by runtimes and then took the average set the runtime as the mean value.
df_combined.nunique()

release_date                  1088
movie_title                   2207
release_year                    62
release_month                   12
adjusted_production_budget    1289
adjusted_worldwide_gross      2059
net_profit                    2136
runtime_minutes                170
genres                         335
dtype: int64

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [91]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***